In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import scipy.optimize as sco
import numpy.random as npr

In [2]:
xls = pd.ExcelFile("HW2_Due20180303.xlsx")
sp500 = pd.read_excel(xls, "SP500Stocks")
spx_dxy_mr = pd.read_excel(xls, "SPX_DXY_MonthlyReturns")

In [3]:
sp500

,,,,Monthly Returns (%),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209
NAME,TICKER,SECTOR,Market Cap ($ Mil),2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,2002-02-28 00:00:00,2002-03-31 00:00:00,2002-04-30 00:00:00,...,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00
3m Co,MMM,INDU,110949,-1.9451,-6.44653,-5.4759,6.07728,10.3396,3.16801,-6.09837,6.80503,-2.47521,9.38125,...,-6.78936,-11.4479,2.14966,-0.258595,7.93032,0.00681024,-0.0996703,-9.70525,10.0302,-8.35896
Abbott Labs,ABT,HLTH,127036,12.113,-7.25895,4.32606,2.59997,3.81273,1.36348,3.88689,-1.99304,-6.98497,3.0142,...,-0.679629,-2.52551,5.84892,-0.877646,7.93986,1.98346,9.75468,-5.64553,7.41221,-2.32271
Abbvie Inc,ABBV,HLTH,138674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-18.2854,3.07046,2.47537,-6.35735,0.546199,4.0659,-1.4586,-16.8175,21.0919,-2.2064
Abiomed Inc,ABMD,HLTH,14640,-19.4232,-1.31579,-6.82667,24.4419,-12.7415,-16.6052,-6.13148,-33.1313,11.7825,-36.8468,...,8.50548,3.4228,26.6456,7.32277,-13.3284,14.6814,10.6178,-24.1356,-2.49707,-2.2965
Accenture Plc Ireland,ACN,INFT,89887,NaN,-0.40107,-14.4295,37.8039,28.6283,19.115,-4.23477,1.59038,1.94731,-19.7004,...,-4.66432,-0.615572,3.00264,5.04044,-2.60404,6.11307,0.668382,-6.54039,4.37762,-14.2901
Activision Blizzard Inc,ATVI,TCOM,35535,-13.5777,9.22631,-26.5328,32.8087,3.11369,4.66793,1.03651,8.6758,4.44818,5.52993,...,-7.2779,-1.64535,6.87257,7.62938,-3.79976,-1.79792,15.3815,-16.9972,-27.7626,-6.63582
Adobe Sys Inc,ADBE,INFT,110435,-20.2341,-10.3492,-28.6168,10.0918,21.5153,-3.17198,8.53469,7.95242,10.7832,-0.819307,...,3.32329,2.55462,12.4909,-2.1943,0.356837,7.69574,2.44394,-8.96091,2.0874,-9.82502
Advance Auto Parts,AAP,DSCR,11478,NaN,NaN,NaN,NaN,NaN,16.1031,-5.82524,-4.37984,2.67863,28.6963,...,3.81933,-3.4584,12.3801,5.55004,4.0752,16.1444,2.65767,-5.09112,11.2354,-11.3602
Advanced Micro Devic,AMD,INFT,18449,-36.8166,-25.7941,-39.8524,20.7362,37.8049,16.9617,1.19798,-15.8879,8.96296,-23.9973,...,-17.0107,8.25871,26.1949,9.17698,22.2815,37.3159,22.7255,-41.0489,16.9687,-13.3333


In [4]:
spx_dxy_mr = spx_dxy_mr.rename(columns={"S&P - 500 Index": "SP500Index", "US Dollar Index":"USDollarIndex"}) 
spx_dxy_mr.head()

,SP500Index,USDollarIndex
2001-07-31,-0.984416,-1.916799
2001-08-31,-6.260163,-3.208739
2001-09-30,-8.075231,-0.008817
2001-10-31,1.906875,1.278547
2001-11-30,7.670629,1.105694


# Problem1 - A

In [5]:
#rename the columns to date time
for i in range(len(sp500.columns)):
    sp500 = sp500.rename(index=str, columns={sp500.columns[i]:sp500.iloc[0][i].date()})
    

In [6]:
temp1 = sp500.copy()
temp1.reset_index(level=[0,2,3], inplace=True) 
temp1 = temp1.drop(columns=["level_0","level_2", "level_3"])
temp1= temp1[1:]
companyRet = temp1.T

In [7]:
#get compamy return in one dataframe, index is date and columns is tickers
companyRet = companyRet.rename(columns={"GOOG.L":"GOOGL"})
companyRet.head()

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AAP,AMD,AES,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
2001-07-31,-1.9451,12.113,NaN,-19.4232,NaN,-13.5777,-20.2341,NaN,-36.8166,-11.0337,...,-2.8169,NaN,-4.07711,-16.6145,-3.00679,NaN,4.21461,NaN,-0.932129,NaN
2001-08-31,-6.44653,-7.25895,NaN,-1.31579,-0.40107,9.22631,-10.3492,NaN,-25.7941,-13.5248,...,9.04362,NaN,1.70733,15.2882,-2.4,NaN,-6.842,-4.8951,-1.69851,NaN
2001-09-30,-5.4759,4.32606,NaN,-6.82667,-14.4295,-26.5328,-28.6168,NaN,-39.8524,-61.2923,...,24.3487,NaN,4.10613,-15.7609,-39.7285,NaN,-7.97755,2.02206,-6.28727,NaN
2001-10-31,6.07728,2.59997,NaN,24.4419,37.8039,32.8087,10.0918,NaN,20.7362,8.03432,...,-0.427533,NaN,0.461629,-9.67742,29.2818,NaN,28.9903,11.3874,-10.6969,NaN
2001-11-30,10.3396,3.81273,NaN,-12.7415,28.6283,3.11369,21.5153,NaN,37.8049,19.278,...,1.67452,NaN,-3.42982,20.0002,18.7048,NaN,-6.20615,4.36752,1.16457,NaN


## 1. Equity market factor. Using SP500 index return as equity market return proxy. (Data see SPX_DXY_MonthlyReturns)

## 2. US Dollar factor. Using  DXY Index returns as proxy. (Data see SPX_DXY_MonthlyReturns)

In [8]:
equity_dollar_fators=pd.DataFrame()
for ticker in companyRet.columns:
    temp = pd.concat([companyRet[ticker], spx_dxy_mr], axis=1)
    temp.dropna(inplace=True)
    x = temp[['SP500Index','USDollarIndex']]
    x = sm.add_constant(x)
    y = temp[ticker].astype(float)
    stats = sm.OLS(y,x).fit()      # run the regression to compute the beta for equity factor and dollar factor
    coeff = stats.params
    equity_dollar_fators[ticker] = coeff
equity_dollar_fators = equity_dollar_fators.T
equity_dollar_fators.head()

,const,SP500Index,USDollarIndex
MMM,0.411085,0.897279,0.137059
ABT,0.635259,0.501031,-0.096830
ABBV,0.662926,1.450096,-0.556901
ABMD,1.770226,1.263956,0.059849
ACN,0.772685,1.152855,0.193686


## 3. Sector factors

In [9]:
temp2 = sp500.copy()
temp2.reset_index(level=[0,1,3], inplace=True) 
temp2 = temp2.drop(columns=["level_0","level_1", "level_3"])
temp2 = temp2[1:]
temp2 = temp2.T
temp2.head()

,INDU,HLTH,HLTH,HLTH,INFT,TCOM,INFT,DSCR,INFT,UTIL,...,FINA,DSCR,UTIL,INFT,INFT,INDU,DSCR,HLTH,FINA,HLTH
2001-07-31,-1.9451,12.113,NaN,-19.4232,NaN,-13.5777,-20.2341,NaN,-36.8166,-11.0337,...,-2.8169,NaN,-4.07711,-16.6145,-3.00679,NaN,4.21461,NaN,-0.932129,NaN
2001-08-31,-6.44653,-7.25895,NaN,-1.31579,-0.40107,9.22631,-10.3492,NaN,-25.7941,-13.5248,...,9.04362,NaN,1.70733,15.2882,-2.4,NaN,-6.842,-4.8951,-1.69851,NaN
2001-09-30,-5.4759,4.32606,NaN,-6.82667,-14.4295,-26.5328,-28.6168,NaN,-39.8524,-61.2923,...,24.3487,NaN,4.10613,-15.7609,-39.7285,NaN,-7.97755,2.02206,-6.28727,NaN
2001-10-31,6.07728,2.59997,NaN,24.4419,37.8039,32.8087,10.0918,NaN,20.7362,8.03432,...,-0.427533,NaN,0.461629,-9.67742,29.2818,NaN,28.9903,11.3874,-10.6969,NaN
2001-11-30,10.3396,3.81273,NaN,-12.7415,28.6283,3.11369,21.5153,NaN,37.8049,19.278,...,1.67452,NaN,-3.42982,20.0002,18.7048,NaN,-6.20615,4.36752,1.16457,NaN


In [10]:
temp3 = sp500.copy()
temp3.reset_index(level=[0,2,3], inplace=True) 
temp3 = temp3[["level_2"]]
temp3 = temp3.rename(columns={'level_2':'SECTOR'})
temp3.index.name = 'TICKER'
temp3 = temp3[1:]
temp3 = temp3.rename(index={"GOOG.L":"GOOGL"})
#temp3.loc['GOOGL']['SECTOR']

In [11]:
sectorsList = set(temp2.columns)
sectorsList.remove('UTIL')
sectorsList

{'DSCR',
 'ENER',
 'FINA',
 'HLTH',
 'INDU',
 'INFT',
 'MATS',
 'REAL',
 'STPL',
 'TCOM'}

In [12]:
# 1 for company's sector, 0 for the rest
sectorFactors = pd.DataFrame(data=0, index=companyRet.columns, columns=sectorsList)
for name in companyRet.columns:
    sectorFactors.loc[name][temp3.loc[name]['SECTOR']] = 1
    
sectorFactors.head(10)

,REAL,INDU,DSCR,MATS,STPL,TCOM,HLTH,ENER,INFT,FINA
MMM,0,1,0,0,0,0,0,0,0,0
ABT,0,0,0,0,0,0,1,0,0,0
ABBV,0,0,0,0,0,0,1,0,0,0
ABMD,0,0,0,0,0,0,1,0,0,0
ACN,0,0,0,0,0,0,0,0,1,0
ATVI,0,0,0,0,0,1,0,0,0,0
ADBE,0,0,0,0,0,0,0,0,1,0
AAP,0,0,1,0,0,0,0,0,0,0
AMD,0,0,0,0,0,0,0,0,1,0
AES,0,0,0,0,0,0,0,0,0,0


## 4. Size factor

In [13]:
# get the market cap of each company
sp500.reset_index(level=[0,1,2,3], inplace=True)
for i in range(4):
    sp500 = sp500.rename(index=str, columns={sp500.columns[i]:sp500.iloc[0][i]})
sp500 = sp500[1:]
sp500 = sp500.rename(columns={'Market Cap ($ Mil)':'MarketCap'})

In [14]:
MC = sp500[['TICKER', 'MarketCap']]
MC['MarketCap'] = MC['MarketCap'].astype(int)
MC_mean = np.log(MC['MarketCap']).mean()
MC_std = np.log(MC['MarketCap']).std()
MC['NorMC'] = (np.log(MC['MarketCap']) - MC_mean) / MC_std    # compute the normalized market cap
MC.set_index('TICKER', inplace=True)
MC = MC.rename(index={"GOOG.L":"GOOGL"})
MC.head()

/Users/keyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/keyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,MarketCap,NorMC
TICKER,,
MMM,110949,1.544283
ABT,127036,1.676796
ABBV,138674,1.762583
ABMD,14640,-0.437856
ACN,89887,1.338253


# Problem1 - C
## (1) all factor portfolios are long-short neutral portfolio, i.e., the total weights sum to 0.

In [15]:
# combined equity factor, dollar factor, sector factor and market cap factor in one dateframe, index is ticker
# This is the result after we run the first regression
beta = pd.concat([equity_dollar_fators, sectorFactors,MC['NorMC']],axis=1)
beta['const'] = 1
beta.head()

,const,SP500Index,USDollarIndex,REAL,INDU,DSCR,MATS,STPL,TCOM,HLTH,ENER,INFT,FINA,NorMC
TICKER,,,,,,,,,,,,,,
MMM,1,0.897279,0.137059,0,1,0,0,0,0,0,0,0,0,1.544283
ABT,1,0.501031,-0.096830,0,0,0,0,0,0,1,0,0,0,1.676796
ABBV,1,1.450096,-0.556901,0,0,0,0,0,0,1,0,0,0,1.762583
ABMD,1,1.263956,0.059849,0,0,0,0,0,0,1,0,0,0,-0.437856
ACN,1,1.152855,0.193686,0,0,0,0,0,0,0,0,1,0,1.338253


In [16]:
# Applied formule to compute the weight(M) matrix
M = np.dot(np.linalg.inv(np.dot(beta.T, beta)), beta.T) 
M.sum(axis=1).round(12) #each row sums to zero excecpt the constant row

array([ 1., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
       -0.])

## (2) for any factor portfolio, it has unit exposure to its own factor, but zero exposure to all other factors in the model. 

In [17]:
print(np.dot(M, beta).round(12))

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0.  0. -0.]
 [-0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  0. -0.]
 [-0. -0.  1. -0.  0. -0.  0. -0. -0. -0.  0. -0. -0.  0.]
 [-0. -0.  0.  1. -0. -0. -0. -0. -0. -0.  0. -0. -0.  0.]
 [ 0.  0. -0.  0.  1.  0. -0. -0. -0.  0.  0. -0. -0.  0.]
 [-0.  0.  0. -0. -0.  1. -0.  0. -0.  0.  0.  0. -0.  0.]
 [ 0.  0.  0.  0.  0. -0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0. -0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -0. -0. -0. -0.  1.  0.  0.  0. -0.  0.]
 [ 0.  0. -0.  0. -0. -0. -0. -0.  0.  1.  0.  0.  0.  0.]
 [-0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  1. -0. -0.  0.]
 [ 0.  0. -0.  0. -0. -0.  0.  0.  0.  0.  0.  1. -0.  0.]
 [-0.  0. -0.  0. -0. -0. -0. -0.  0.  0.  0. -0.  1.  0.]
 [-0.  0.  0. -0. -0.  0. -0. -0. -0. -0.  0.  0.  0.  1.]]


# Problem1 - B
## check for each factor their historical returns are significant or not (based on T-stat).

In [18]:
# run the cross sectional regression to get the factor portfolio
factor_Portfolio=pd.DataFrame()
for i in range(len(companyRet.index)):
    temp = beta.copy()
    temp['Ret'] = companyRet.iloc[i]
    temp.dropna(inplace=True)
    x = temp[['SP500Index','USDollarIndex','TCOM','HLTH','INFT','ENER','STPL','FINA','MATS','DSCR','INDU','REAL','NorMC']]
    x = sm.add_constant(x)
    y = temp['Ret'].astype(float)
    stats = sm.OLS(y,x).fit()
    coeff = stats.params
    factor_Portfolio[companyRet.index[i]] = coeff
factor_Portfolio = factor_Portfolio.T
factor_Portfolio.head()

,const,SP500Index,USDollarIndex,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL,NorMC
2001-07-31,0.493914,-6.797837,1.124098,5.301104,4.779400,7.541634,5.275471,6.902788,7.414378,8.853866,11.926845,9.694429,3.536493,-0.892460
2001-08-31,4.141394,-6.540678,-3.810894,-0.235306,0.649443,-0.324320,-3.381388,1.078840,-0.209966,1.955901,-1.093426,0.305834,1.765530,-1.026595
2001-09-30,2.368606,-20.742234,-4.441778,5.601557,8.865722,8.744905,4.066462,6.084597,17.936203,10.429827,6.096653,9.967786,12.672593,-0.088294
2001-10-31,-2.773098,9.057487,5.576869,4.222647,3.403955,8.651703,7.901440,0.084666,-9.500843,-1.952621,0.271634,-3.606698,-6.572382,-1.125156
2001-11-30,-4.618825,11.184601,5.869029,-0.249032,0.079082,2.952645,-8.529357,2.489815,-0.774401,5.062111,4.555630,2.138210,0.840661,1.016303


In [19]:
stats= factor_Portfolio.describe()
stats.loc['stderr'] = stats.loc['std'] / np.sqrt(stats.loc['count'])
stats.loc['tstat']  = stats.loc['mean'] / stats.loc['stderr']
stats.loc['tstat']

const            2.750728
SP500Index       0.616690
USDollarIndex    0.024184
TCOM             0.252914
HLTH             1.377913
INFT             1.031994
ENER             0.419303
STPL             0.461590
FINA            -0.475549
MATS             0.866117
DSCR             0.942758
INDU             0.699691
REAL             1.195410
NorMC            0.870056
Name: tstat, dtype: float64

### Base on the T-stats we got, we can see that none of factors is significant

# Problem 2 - A

In [20]:
sector_ret = factor_Portfolio[['TCOM','HLTH','INFT','ENER','STPL','FINA','MATS','DSCR','INDU','REAL']]

sector_ret.head()

,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL
2001-07-31,5.301104,4.779400,7.541634,5.275471,6.902788,7.414378,8.853866,11.926845,9.694429,3.536493
2001-08-31,-0.235306,0.649443,-0.324320,-3.381388,1.078840,-0.209966,1.955901,-1.093426,0.305834,1.765530
2001-09-30,5.601557,8.865722,8.744905,4.066462,6.084597,17.936203,10.429827,6.096653,9.967786,12.672593
2001-10-31,4.222647,3.403955,8.651703,7.901440,0.084666,-9.500843,-1.952621,0.271634,-3.606698,-6.572382
2001-11-30,-0.249032,0.079082,2.952645,-8.529357,2.489815,-0.774401,5.062111,4.555630,2.138210,0.840661


In [21]:
#equlibrium expected monthly returns (Pi) are assumed to equal historical average monthly returns.
equl_exp_ret = sector_ret.mean()
equl_exp_ret

TCOM    0.079243
HLTH    0.395389
INFT    0.317072
ENER    0.174072
STPL    0.115598
FINA   -0.164984
MATS    0.267166
DSCR    0.320013
INDU    0.210220
REAL    0.372071
dtype: float64

In [22]:
sigma = sector_ret.cov() 
sigma

,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL
TCOM,20.615376,12.791685,14.765621,13.785335,12.478466,14.198339,14.484290,15.733694,13.632703,7.201409
HLTH,12.791685,17.291191,12.961172,9.377191,11.217920,14.549189,12.441096,14.154106,13.549515,9.355800
INFT,14.765621,12.961172,19.823554,13.375686,10.450357,12.689078,14.474817,14.189648,13.802750,6.335535
ENER,13.785335,9.377191,13.375686,36.192545,8.080287,10.954427,15.078956,9.946333,13.128805,1.998414
STPL,12.478466,11.217920,10.450357,8.080287,13.170658,13.268221,11.213943,14.035625,12.318443,8.084874
FINA,14.198339,14.549189,12.689078,10.954427,13.268221,25.276270,15.935734,18.278457,17.220080,11.483809
MATS,14.484290,12.441096,14.474817,15.078956,11.213943,15.935734,19.981594,16.192706,16.538627,7.954870
DSCR,15.733694,14.154106,14.189648,9.946333,14.035625,18.278457,16.192706,24.196527,17.463108,12.384759
INDU,13.632703,13.549515,13.802750,13.128805,12.318443,17.220080,16.538627,17.463108,18.956306,9.072141
REAL,7.201409,9.355800,6.335535,1.998414,8.084874,11.483809,7.954870,12.384759,9.072141,20.344080


In [23]:
risk_free_rate = 0 # assume risk free rate is 0
def portfolio_calculation(weight, expect_return, cov_matrix): 
    '''
    This is function in order to calculate the portfolio return, volatility and Sharpe Ratio
    '''
    w = np.array(weight)

    prof_return = np.sum(expect_return * w* 12)      #annualized return
    prof_volatility = np.sqrt(np.dot(w.T, np.dot(cov_matrix * 12, w)))  #annualized cov
    sharpe_ratio = (prof_return - risk_free_rate) / prof_volatility     
    return np.array([prof_return, prof_volatility, sharpe_ratio])

# minimize negative return = maxmize positive return
def returns(weight,expect_return, cov_matrix):
    return -portfolio_calculation(weight,expect_return, cov_matrix)[0]

def volatility(weight,expect_return, cov_matrix):
    return portfolio_calculation(weight,expect_return, cov_matrix)[1]

# minimize negative Sharpe ratio = maximize positive Sharpe ratio
def sharpe_ratio(weight,expect_return, cov_matrix):
    return -portfolio_calculation(weight,expect_return, cov_matrix)[2]


In [24]:
def results(expect_return, cov_matrix):
    '''
    return the weights of portfolio of minimum volatility, maximum return and maximum sharpe ratio
    '''
    num_universe = 10 #10 sectors
    arg = (expect_return, cov_matrix)
    #only constraint is annual standard deviation no greater than 10
    consts = ({'type': 'eq', 'fun': lambda x: volatility(x,expect_return, cov_matrix) - 9.999} ) 
    bnds = () 
    
    min_volatility = sco.minimize(volatility, num_universe*[1./num_universe,],args= arg ,
                        method='SLSQP', bounds=bnds, constraints=consts)
    max_ret = sco.minimize(returns, num_universe*[1./num_universe,],args= arg ,
                        method='SLSQP', bounds=bnds, constraints=consts)
    max_sr = sco.minimize(sharpe_ratio, num_universe*[1./num_universe,],args= arg ,
                        method='SLSQP', bounds=bnds, constraints=consts)

    return min_volatility['x'].round(4), max_ret['x'].round(4), max_sr['x'].round(4)

In [25]:
def display_result(expect_return, cov_matrix):
    '''
    Display all weights clearly
    '''

    weights_min_vol, weights_max_ret, weights_max_sr = results(expect_return, cov_matrix)

    print("-"*50)
    VOL = volatility(weights_max_ret,expect_return, cov_matrix)
    max_return = -returns(weights_max_ret,expect_return, cov_matrix)
    sr = -sharpe_ratio(weights_max_ret,expect_return, cov_matrix)    
    maxReturn_allocation = pd.DataFrame(weights_max_ret, index=cov_matrix.columns, columns=['Weights'])
    print("Maximum Return Portfolio Allocation\n")
    print("Volatility: {}%".format(round(VOL,4)))
    print("Portfoilo Return: {}%".format(round(max_return, 3)))
    print("Maximum Sharpe Ratio: ", round(sr, 2))
    print("-"*50)
    print(maxReturn_allocation)     
    print("-"*50)
    print('\n')
    

In [26]:
display_result(equl_exp_ret, sigma)

--------------------------------------------------
Maximum Return Portfolio Allocation

Volatility: 9.9992%
Portfoilo Return: 7.381%
Maximum Sharpe Ratio:  0.74
--------------------------------------------------
      Weights
TCOM  -0.3413
HLTH   0.6041
INFT   0.0998
ENER   0.0501
STPL  -0.1207
FINA  -0.7348
MATS   0.2932
DSCR   0.3232
INDU  -0.0632
REAL   0.2343
--------------------------------------------------




# Problem 2 - B
## (1) show the updated factor expected return distribution and compare it with the original one

In [27]:
views = np.zeros(shape=(3,10))
#Sector ENER will have return of 0.20% in next month.
ENER_index = sigma.columns.get_loc('ENER')
views[0][ENER_index] = 1 
#Sector FINA will outperform Sector INDU by 0.05% in next month.
FINA_index = sigma.columns.get_loc('FINA')
INDU_index = sigma.columns.get_loc('INDU')
views[1][FINA_index] = 1 
views[1][INDU_index] = -1
#Sector INFT and Sector HLTH will outperform Sector REAL and Sector STPL by 0.30% in next month. 
#(Assuming equal weighting on both long and short legs)
INFT_index = sigma.columns.get_loc('INFT')
HLTH_index = sigma.columns.get_loc('HLTH')
REAL_index = sigma.columns.get_loc('REAL')
STPL_index = sigma.columns.get_loc('STPL')
views[2][INFT_index] = 0.5
views[2][HLTH_index] = 0.5
views[2][REAL_index] = -0.5
views[2][STPL_index] = -0.5
views

array([[ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. , -1. ,  0. ],
       [ 0. ,  0.5,  0.5,  0. , -0.5,  0. ,  0. ,  0. ,  0. , -0.5]])

In [28]:
pd.DataFrame(data=views, index=['view1', 'view2', 'view3'], columns=sigma.columns)

,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL
view1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
view2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0
view3,0.0,0.5,0.5,0.0,-0.5,0.0,0.0,0.0,0.0,-0.5


In [29]:
tau = 0.05  
omega = np.zeros(shape=(3,3)) 
for i in range(3):
    #Assuming view uncertainty for each view portfolio equals the historical variance of each view portfolio. 
    #(omega1=p1*Sigma*p1').
    omega[i][i] = np.dot(np.dot(views[i], sigma), views[i].T) #*tau 
omega

array([[36.19254486,  0.        ,  0.        ],
       [ 0.        ,  9.79241654,  0.        ],
       [ 0.        ,  0.        ,  9.5005874 ]])

In [30]:
C = tau * sigma
C

,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL
TCOM,1.030769,0.639584,0.738281,0.689267,0.623923,0.709917,0.724215,0.786685,0.681635,0.360070
HLTH,0.639584,0.864560,0.648059,0.468860,0.560896,0.727459,0.622055,0.707705,0.677476,0.467790
INFT,0.738281,0.648059,0.991178,0.668784,0.522518,0.634454,0.723741,0.709482,0.690137,0.316777
ENER,0.689267,0.468860,0.668784,1.809627,0.404014,0.547721,0.753948,0.497317,0.656440,0.099921
STPL,0.623923,0.560896,0.522518,0.404014,0.658533,0.663411,0.560697,0.701781,0.615922,0.404244
FINA,0.709917,0.727459,0.634454,0.547721,0.663411,1.263814,0.796787,0.913923,0.861004,0.574190
MATS,0.724215,0.622055,0.723741,0.753948,0.560697,0.796787,0.999080,0.809635,0.826931,0.397743
DSCR,0.786685,0.707705,0.709482,0.497317,0.701781,0.913923,0.809635,1.209826,0.873155,0.619238
INDU,0.681635,0.677476,0.690137,0.656440,0.615922,0.861004,0.826931,0.873155,0.947815,0.453607
REAL,0.360070,0.467790,0.316777,0.099921,0.404244,0.574190,0.397743,0.619238,0.453607,1.017204


In [31]:
#active views return vector
Q = np.zeros(shape=(3,1))
Q[0][0] = 0.2
Q[1][0] = 0.05
Q[2][0] = 0.3
Q

array([[0.2 ],
       [0.05],
       [0.3 ]])

In [32]:
H = np.dot(np.dot(views.T, np.linalg.inv(omega)), views) + np.linalg.inv(C)
H 

array([[ 3.40824427, -0.32243088, -0.91985205, -0.38122621, -1.6656601 ,
        -0.07657456, -0.63848333, -0.65359434,  0.92594366,  0.20553535],
       [-0.32243088,  3.75607442, -0.98112248,  0.06886682, -0.98885753,
        -0.49753054,  0.25617074,  0.23461795, -0.88004072, -0.50763149],
       [-0.91985205, -0.98112248,  3.09772688, -0.15677321,  0.14035036,
         0.36612108, -0.74257935, -0.20049905, -0.44300237,  0.15856364],
       [-0.38122621,  0.06886682, -0.15677321,  0.96094463,  0.15693475,
         0.0300401 , -0.4468827 ,  0.33263351, -0.4205374 ,  0.14086973],
       [-1.6656601 , -0.98885753,  0.14035036,  0.15693475,  6.2553807 ,
        -0.5140724 ,  0.44340186, -0.99743092, -1.29242422, -0.18386892],
       [-0.07657456, -0.49753054,  0.36612108,  0.0300401 , -0.5140724 ,
         2.62207632, -0.39979525, -0.33571595, -1.15710886, -0.24807038],
       [-0.63848333,  0.25617074, -0.74257935, -0.4468827 ,  0.44340186,
        -0.39979525,  4.57304203, -0.32356886

In [33]:
pi = pd.DataFrame(data=equl_exp_ret, columns=['Exp_Ret'])
nu = np.dot(np.linalg.inv(H), np.dot(np.dot(views.T, np.linalg.inv(omega)), Q) + np.dot(np.linalg.inv(C), pi))
nu

array([[ 0.08463001],
       [ 0.40239833],
       [ 0.3228475 ],
       [ 0.17678517],
       [ 0.11802663],
       [-0.14671688],
       [ 0.27010678],
       [ 0.32294523],
       [ 0.20988992],
       [ 0.37104491]])

## (2) make observations on how active views impact updated returns.

### Base on active views, update return of ENER is increased. FINA is increased compare to INDU is decreased. INFT and HLTH both increased. REAL is decreased, but STPL is increased. The update return of INFT + HLTH > REAL + STPL. Since our views' return are small, amount of increasing or decreasing is small. In conclusion, positive active view will increase expected return and negative view will decrease expected return

In [34]:
pi['Update_Ret'] = nu
pi

,Exp_Ret,Update_Ret
TCOM,0.079243,0.084630
HLTH,0.395389,0.402398
INFT,0.317072,0.322847
ENER,0.174072,0.176785
STPL,0.115598,0.118027
FINA,-0.164984,-0.146717
MATS,0.267166,0.270107
DSCR,0.320013,0.322945
INDU,0.210220,0.209890
REAL,0.372071,0.371045


In [35]:
update_cov = np.linalg.inv(H) + sigma
update_cov

,TCOM,HLTH,INFT,ENER,STPL,FINA,MATS,DSCR,INDU,REAL
TCOM,21.629879,13.418036,15.484367,14.436307,13.094777,14.895961,15.191319,16.510384,14.299940,7.565421
HLTH,13.418036,18.144007,13.591827,9.817045,11.773413,15.266414,13.049513,14.854406,14.215899,9.829365
INFT,15.484367,13.591827,20.787118,14.000196,10.965729,13.313599,15.177749,14.888849,14.475247,6.663758
ENER,14.436307,9.817045,14.000196,37.905877,8.465330,11.478299,15.791073,10.419056,13.748820,2.104397
STPL,13.094777,11.773413,10.965729,8.465330,13.824660,13.923850,11.766619,14.731886,12.927683,8.487718
FINA,14.895961,15.266414,13.313599,11.478299,13.923850,26.515913,16.721611,19.183282,18.074089,12.053751
MATS,15.191319,13.049513,15.177749,15.791073,11.766619,16.721611,20.962127,16.991758,17.349623,8.356933
DSCR,16.510384,14.854406,14.888849,10.419056,14.731886,19.183282,16.991758,25.399475,18.327387,13.003607
INDU,14.299940,14.215899,14.475247,13.748820,12.927683,18.074089,17.349623,18.327387,19.890034,9.529580
REAL,7.565421,9.829365,6.663758,2.104397,8.487718,12.053751,8.356933,13.003607,9.529580,21.349443


## (3)  Re-run the optimization as in A, but with the new return vector and covariance matrix. Please compare the updated optimal portfolio (the weight vector) with the solution from A.

### After we update expected return base on active view, the portfolio return is much higher than before under same standard divation constraint. Two portfolio varaince are almost the same. And the weights of sectors with positive active view have increased.

In [36]:
display_result(nu, update_cov)

--------------------------------------------------
Maximum Return Portfolio Allocation

Volatility: 9.9995%
Portfoilo Return: 22.961%
Maximum Sharpe Ratio:  2.3
--------------------------------------------------
      Weights
TCOM  -0.0522
HLTH   0.0610
INFT   0.1410
ENER   0.1143
STPL   0.5985
FINA  -0.0931
MATS   0.0881
DSCR  -0.2359
INDU  -0.0325
REAL   0.3083
--------------------------------------------------




In [37]:
print("Before Update the Active Views \n")
display_result(equl_exp_ret, sigma)

Before Update the Active Views 

--------------------------------------------------
Maximum Return Portfolio Allocation

Volatility: 9.9992%
Portfoilo Return: 7.381%
Maximum Sharpe Ratio:  0.74
--------------------------------------------------
      Weights
TCOM  -0.3413
HLTH   0.6041
INFT   0.0998
ENER   0.0501
STPL  -0.1207
FINA  -0.7348
MATS   0.2932
DSCR   0.3232
INDU  -0.0632
REAL   0.2343
--------------------------------------------------


